## Problem Statement

To do

## Model
Define $S$ as the set of studios, $T$ as the sets of timeslots, $D$ as the set of days, $P$ as the sets of programs, and $C$ as the set of coaches. Also, define the following: 
- $q_{pc}$: the qualification of coach $c$ to teach program $p$ ($1$ if qualified, $0$ otherwise).
- $p_{stdpc}$: the profit when coach $c$ teaches program $p$ at studio $s$ on day $d$ at time $t$.

Let $x_{stdpc}$ be defined such that if $x_{stdpc} = 1$, then coach $c$ is scheduled to teach program $p$ at studio $s$ on day $d$ at time $t$. We formulate as follows.

Let $x_{stdpc}$ be the scheduling of coach $c$ to teach program $p$ at studio $s$ on day $d$ at time $t$. If the value is $1$ then the schedule exist, $0$ otherwise. We formulate as follows.


$$
\begin{alignat*}{3}
\text{maximize  }  & \sum_{s \in S} \sum_{t \in T} \sum_{d \in D} \sum_{p \in P} \sum_{c \in C} p_{stdpc} x_{stdpc} && \\
\text{subject to  }
& \sum_{p \in P} \sum_{c \in C} x_{stdpc} = 1,
&& \qquad \forall S \in S, \forall t \in T , \forall d \in D  \\
& x_{stdpc} = 0
&& \qquad \forall p \in P, \forall c \in C \text{ where } q_{pc} = 0\\
& 0 \le x_{stdpc} \le 1,
&& \qquad \forall s \in S, \forall t \in T, \forall d \in D, \forall p \in P, \forall c \in C
\end{alignat*}
$$

## Import

In [1]:
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po

## Define Data

In [2]:
studios = {'S1'}
timeslots = {'T1'}
days = {'D1', 'D2', 'D3'}
programs = {'P1', 'P2', 'P3'}
coaches = {'C1', 'C2', 'C3'}

q = {
    ('P1', 'C1'): 1,
    ('P1', 'C2'): 1,
    ('P1', 'C3'): 1,
    ('P2', 'C1'): 1,
    ('P2', 'C2'): 1,
    ('P2', 'C3'): 1,
    ('P3', 'C1'): 1,
    ('P3', 'C2'): 1,
    ('P3', 'C3'): 1,
}

p = {
    ('S1', 'T1', 'D1', 'P1', 'C1'): 1,
    ('S1', 'T1', 'D1', 'P1', 'C2'): 1,
    ('S1', 'T1', 'D1', 'P1', 'C3'): 1,
    ('S1', 'T1', 'D1', 'P2', 'C1'): 1,
    ('S1', 'T1', 'D1', 'P2', 'C2'): 1,
    ('S1', 'T1', 'D1', 'P2', 'C3'): 1,
    ('S1', 'T1', 'D1', 'P3', 'C1'): 1,
    ('S1', 'T1', 'D1', 'P3', 'C2'): 1,
    ('S1', 'T1', 'D1', 'P3', 'C3'): 1,
    ('S1', 'T1', 'D2', 'P1', 'C1'): 1,
    ('S1', 'T1', 'D2', 'P1', 'C2'): 1,
    ('S1', 'T1', 'D2', 'P1', 'C3'): 1,
    ('S1', 'T1', 'D2', 'P2', 'C1'): 1,
    ('S1', 'T1', 'D2', 'P2', 'C2'): 1,
    ('S1', 'T1', 'D2', 'P2', 'C3'): 1,
    ('S1', 'T1', 'D2', 'P3', 'C1'): 1,
    ('S1', 'T1', 'D2', 'P3', 'C2'): 1,
    ('S1', 'T1', 'D2', 'P3', 'C3'): 1,
    ('S1', 'T1', 'D3', 'P1', 'C1'): 1,
    ('S1', 'T1', 'D3', 'P1', 'C2'): 1,
    ('S1', 'T1', 'D3', 'P1', 'C3'): 1,
    ('S1', 'T1', 'D3', 'P2', 'C1'): 1,
    ('S1', 'T1', 'D3', 'P2', 'C2'): 1,
    ('S1', 'T1', 'D3', 'P2', 'C3'): 1,
    ('S1', 'T1', 'D3', 'P3', 'C1'): 1,
    ('S1', 'T1', 'D3', 'P3', 'C2'): 1,
    ('S1', 'T1', 'D3', 'P3', 'C3'): 1,
}

## Implement

In [6]:
# Model
model = pe.ConcreteModel()

# Sets
model.studios = pe.Set(initialize=studios)
model.timeslots = pe.Set(initialize=timeslots)
model.days = pe.Set(initialize=days)
model.programs = pe.Set(initialize=programs)
model.coaches = pe.Set(initialize=coaches)

# Parameters
model.q = pe.Param(model.programs, model.coaches, initialize=q, default=-1000)
model.p = pe.Param(model.studios, model.timeslots, model.days, model.programs, model.coaches, initialize=p, default=-1000)

# Decision variables
model.x = pe.Var(model.studios, model.timeslots, model.days, model.programs, model.coaches, domain=pe.Boolean)

# Objective function
expr = sum(model.p[s, t, d, p, c] * model.x[s, t, d, p, c]
        for s in model.studios 
        for t in model.timeslots
        for d in model.days
        for p in model.programs
        for c in model.coaches)
model.objective = pe.Objective(sense=pe.maximize, expr=expr)

source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo
data source (type: set).  This WILL potentially lead to nondeterministic
behavior in Pyomo
source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo
source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo
source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo


In [ ]:
model.coach_qualifications = pe.ConstraintList()
for w in model.workers:
    lhs = sum(model.c[w, t] * model.x[w, t] for t in model.tasks)
    rhs = model.max_hours
    model.coach_qualifications.add(lhs <= rhs)

## Solve and Postprocess

In [ ]:
solver = po.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

In [ ]:
df = pd.DataFrame(index=pd.MultiIndex.from_tuples(model.x, names=['w', 't']))
df['x'] = [pe.value(model.x[key]) for key in df.index]
df['c'] = [model.c[key] for key in df.index]

In [ ]:
(df['c'] * df['x']).unstack('t')

In [ ]:
(df['c'] * df['x']).groupby('w').sum().to_frame()

In [ ]:
df['x'].groupby('t').sum().to_frame().T